### Import libraries

In [53]:
import pandas as pd
import datetime
import json
from transliterate import get_translit_function

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)

### Preprocess BoardingData.csv

In [54]:
boarding_data = pd.read_csv("data/BoardingData.csv", delimiter=";")
boarding_data.head(5)
boarding_data.info()
boarding_data.describe()


,PassengerFirstName,PassengerSecondName,PassengerLastName,PassengerSex,PassengerBirthDate,PassengerDocument,BookingCode,TicketNumber,Baggage,FlightDate,FlightTime,FlightNumber,CodeShare,Destination
0,SAVELII,VIKTOROVICH,RUSANOV,Male,03/10/1983,2879 096860,FRNINO,6625956945991971,Transit,2017-03-22,06:05,SU1369,Own,Moscow
1,LEV,MARKOVICH,ISAEV,Male,12/13/1975,1788 173211,Not presented,1643715499224676,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
2,NIKOLAI,I.,NIKOLSKII,Male,12/26/1990,4396 926588,VWNYGF,6247422701565929,Transit,2017-03-18,22:10,SU1180,Own,Volgograd
3,ANATOLII,PETROVICH,SHILOV,Male,05/24/1997,2595 919752,WQFFUE,Not presented,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
4,MIROSLAVA,VIACHESLAVOVNA,SEMENOVA,Female,01/31/1976,6775 516990,Not presented,Not presented,Registered,2017-03-12,11:45,SU6284,Own,Moscow


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155147 entries, 0 to 155146
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   PassengerFirstName   155147 non-null  object
 1   PassengerSecondName  155147 non-null  object
 2   PassengerLastName    155147 non-null  object
 3   PassengerSex         155147 non-null  object
 4   PassengerBirthDate   155147 non-null  object
 5   PassengerDocument    155147 non-null  object
 6   BookingCode          155147 non-null  object
 7   TicketNumber         155147 non-null  object
 8   Baggage              155147 non-null  object
 9   FlightDate           155147 non-null  object
 10  FlightTime           155147 non-null  object
 11  FlightNumber         155147 non-null  object
 12  CodeShare            155147 non-null  object
 13  Destination          155147 non-null  object
dtypes: object(14)
memory usage: 16.6+ MB


,PassengerFirstName,PassengerSecondName,PassengerLastName,PassengerSex,PassengerBirthDate,PassengerDocument,BookingCode,TicketNumber,Baggage,FlightDate,FlightTime,FlightNumber,CodeShare,Destination
count,155147,155147,155147,155147,155147,155147,155147,155147,155147,155147,155147,155147,155147,155147
unique,200,214,1006,2,11301,73263,77374,77348,4,366,206,385,2,48
top,NAZAR,A.,CHERNYKH,Female,10/27/1991,1548 113497,Not presented,Not presented,Registered,2017-01-01,06:05,SU4606,Own,Moscow
freq,932,11119,290,77863,45,16,77761,77800,38957,5262,3221,2909,146373,88022


In [55]:
def convert_date_to_iso8601(val: str):
    date = datetime.datetime.strptime(val, "%m/%d/%Y").date()
    return date.strftime('%Y-%m-%d')


boarding_data['PassengerBirthDate'] = boarding_data['PassengerBirthDate'].apply(convert_date_to_iso8601)
boarding_data.rename(
    columns={"FlightDate": "DepartureDate", "FlightTime": "DepartureTime", "Destination": "ArrivalCity",
             "Baggage": "BaggageState"},
    inplace=True)
boarding_data = boarding_data.replace({'Not presented': "NaN"})
boarding_data["CodeShare"].replace({"Own": False, "Operated": True}, inplace=True)
boarding_data = boarding_data.reindex(sorted(boarding_data.columns), axis=1)

boarding_data.head(5)
boarding_data.info()
boarding_data.describe()

,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,TicketNumber
0,Moscow,Transit,FRNINO,False,2017-03-22,06:05,SU1369,1983-03-10,2879 096860,SAVELII,RUSANOV,VIKTOROVICH,Male,6625956945991971
1,Volgograd,Registered,NaN,False,2017-03-18,22:10,SU1180,1975-12-13,1788 173211,LEV,ISAEV,MARKOVICH,Male,1643715499224676
2,Volgograd,Transit,VWNYGF,False,2017-03-18,22:10,SU1180,1990-12-26,4396 926588,NIKOLAI,NIKOLSKII,I.,Male,6247422701565929
3,Volgograd,Registered,WQFFUE,False,2017-03-18,22:10,SU1180,1997-05-24,2595 919752,ANATOLII,SHILOV,PETROVICH,Male,NaN
4,Moscow,Registered,NaN,False,2017-03-12,11:45,SU6284,1976-01-31,6775 516990,MIROSLAVA,SEMENOVA,VIACHESLAVOVNA,Female,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155147 entries, 0 to 155146
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ArrivalCity          155147 non-null  object
 1   BaggageState         155147 non-null  object
 2   BookingCode          155147 non-null  object
 3   CodeShare            155147 non-null  bool  
 4   DepartureDate        155147 non-null  object
 5   DepartureTime        155147 non-null  object
 6   FlightNumber         155147 non-null  object
 7   PassengerBirthDate   155147 non-null  object
 8   PassengerDocument    155147 non-null  object
 9   PassengerFirstName   155147 non-null  object
 10  PassengerLastName    155147 non-null  object
 11  PassengerSecondName  155147 non-null  object
 12  PassengerSex         155147 non-null  object
 13  TicketNumber         155147 non-null  object
dtypes: bool(1), object(13)
memory usage: 15.5+ MB


,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,TicketNumber
count,155147,155147,155147,155147,155147,155147,155147,155147,155147,155147,155147,155147,155147,155147
unique,48,4,77374,2,366,206,385,11301,73263,200,1006,214,2,77348
top,Moscow,Registered,NaN,False,2017-01-01,06:05,SU4606,1991-10-27,1548 113497,NAZAR,CHERNYKH,A.,Female,NaN
freq,88022,38957,77761,146373,5262,3221,2909,45,16,932,290,11119,77863,77800


### Preprocess Sirena-export-fixed.tab

In [56]:
sirena_export = pd.read_fwf("data/Sirena-export-fixed.tab",
                            colspecs=[(0, 60), (60, 72), (72, 84), (84, 96), (96, 108), (108, 120), (120, 132),
                                      (132, 138), (138, 144), (144, 150), (150, 168), (168, 180), (180, 186),
                                      (186, 192), (192, 198), (198, 204), (204, 216), (216, 276), (276, 336)])
sirena_export.head(5)
sirena_export.info()
sirena_export.describe()

,PaxName,PaxBirthDate,DepartDate,DepartTime,ArrivalDate,ArrivalTime,FlightCodeSh,From,Dest,Code,e-Ticket,TravelDoc,Seat,Meal,TrvCls,Fare,Baggage,PaxAdditionalInfo,AgentInfo
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306NO,SVO,OVB,ZBQSPY,7360415302044672,9375 053270,NaN,NaN,J,JGRPGN,0PC,S FF#SU 38116280,Go2See
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323NO,MMK,SVO,MHPBBX,7398421117936516,2244 645520,NaN,KSML,Y,YRSTUQ,NaN,9 FF#FB 284903754,NaN
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481NO,KJA,SVO,REDLVB,5174973140468001,8115 961316,NaN,NaN,Y,YSTNJL,NaN,F,KupiBilet
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,NSJNGQ,5274206497242737,98 6865148,NaN,NaN,J,JFLXLS,0PC,# FF#FB 884556993,Travelgenio
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,VWNYGF,6247422701565929,4396 926588,NaN,NaN,Y,YFLXPG,NaN,FF#SU 183142068,OZON.travel


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155757 entries, 0 to 155756
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   PaxName            155757 non-null  object 
 1   PaxBirthDate       77703 non-null   object 
 2   DepartDate         155757 non-null  object 
 3   DepartTime         155757 non-null  object 
 4   ArrivalDate        155757 non-null  object 
 5   ArrivalTime        155757 non-null  object 
 6   FlightCodeSh       155757 non-null  object 
 7   From               155757 non-null  object 
 8   Dest               155757 non-null  object 
 9   Code               155757 non-null  object 
 10  e-Ticket           155757 non-null  int64  
 11  TravelDoc          155757 non-null  object 
 12  Seat               0 non-null       float64
 13  Meal               58148 non-null   object 
 14  TrvCls             155757 non-null  object 
 15  Fare               155757 non-null  object 
 16  Ba

,e-Ticket,Seat
count,1.557570e+05,0.0
mean,5.010077e+15,NaN
std,2.881055e+15,NaN
min,5.811780e+10,NaN
25%,2.521718e+15,NaN
50%,5.019438e+15,NaN
75%,7.499988e+15,NaN
max,9.999858e+15,NaN


In [57]:
sirena_export[['PassengerLastName', 'PassengerFirstName', 'PassengerSecondName']] = sirena_export['PaxName'].str.split(
    ' ', expand=True)
del sirena_export['PaxName']
sirena_export['FlightNumber'] = sirena_export['FlightCodeSh'].str[0:6]
sirena_export['CodeShare'] = sirena_export['FlightCodeSh'].str[6:]
del sirena_export['FlightCodeSh']
sirena_export.rename(columns={"PaxBirthDate": "PassengerBirthDate", "Code": "BookingCode", "e-Ticket": "TicketNumber",
                              "TravelDoc": "PassengerDocument", "TrvCls": "PassengerClass",
                              "PaxAdditionalInfo": "PassengerAdditionalInfo", "From": "DepartureAirport",
                              "Dest": "ArrivalAirport", "DepartDate": "DepartureDate", "DepartTime": "DepartureTime",
                              "Baggage": "BaggageCount"}, inplace=True)
del sirena_export['Seat']
sirena_export["CodeShare"].replace({"YES": True, "NO": False}, inplace=True)

transliterator_ru = get_translit_function('ru')


def transliterate(val: str):
    return transliterator_ru(val, reversed=True)\
        .replace("'", "")\
        .replace("J", "I")\
        .upper()


sirena_export["PassengerFirstName"] = sirena_export["PassengerFirstName"].apply(transliterate)
sirena_export["PassengerSecondName"] = sirena_export["PassengerSecondName"].apply(transliterate)
sirena_export["PassengerLastName"] = sirena_export["PassengerLastName"].apply(transliterate)

sirena_export = sirena_export.reindex(sorted(sirena_export.columns), axis=1)

sirena_export.head(5)
sirena_export.info()
sirena_export.describe()

,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,BookingCode,CodeShare,DepartureAirport,DepartureDate,DepartureTime,Fare,FlightNumber,Meal,PassengerAdditionalInfo,PassengerBirthDate,PassengerClass,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,TicketNumber
0,Go2See,OVB,2017-05-30,08:05,0PC,ZBQSPY,False,SVO,2017-05-30,00:05,JGRPGN,SU1306,NaN,S FF#SU 38116280,1999-05-15,J,9375 053270,ILDAR,OZEROV,DANIILOVICH,7360415302044672
1,NaN,SVO,2017-12-27,04:40,NaN,MHPBBX,False,MMK,2017-12-27,02:15,YRSTUQ,SU1323,KSML,9 FF#FB 284903754,NaN,Y,2244 645520,SAMIR,KOLOSOV,TAMERLANOVICH,7398421117936516
2,KupiBilet,SVO,2017-09-19,07:45,NaN,REDLVB,False,KJA,2017-09-19,06:40,YSTNJL,SU1481,NaN,F,NaN,Y,8115 961316,SNEZHANA,IGNATOVA,KONSTANTINOVNA,5174973140468001
3,Travelgenio,VOG,2017-03-19,01:05,0PC,NSJNGQ,False,SVO,2017-03-18,22:10,JFLXLS,SU1180,NaN,# FF#FB 884556993,1999-05-02,J,98 6865148,PLATON,ZHAROV,ALBERTOVICH,5274206497242737
4,OZON.travel,VOG,2017-03-19,01:05,NaN,VWNYGF,False,SVO,2017-03-18,22:10,YFLXPG,SU1180,NaN,FF#SU 183142068,1990-12-26,Y,4396 926588,NIKOLAI,NIKOLSKII,IGOREVICH,6247422701565929


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155757 entries, 0 to 155756
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   AgentInfo                143814 non-null  object
 1   ArrivalAirport           155757 non-null  object
 2   ArrivalDate              155757 non-null  object
 3   ArrivalTime              155757 non-null  object
 4   BaggageCount             85064 non-null   object
 5   BookingCode              155757 non-null  object
 6   CodeShare                155757 non-null  bool  
 7   DepartureAirport         155757 non-null  object
 8   DepartureDate            155757 non-null  object
 9   DepartureTime            155757 non-null  object
 10  Fare                     155757 non-null  object
 11  FlightNumber             155757 non-null  object
 12  Meal                     58148 non-null   object
 13  PassengerAdditionalInfo  130530 non-null  object
 14  PassengerBirthDate  

,TicketNumber
count,1.557570e+05
mean,5.010077e+15
std,2.881055e+15
min,5.811780e+10
25%,2.521718e+15
50%,5.019438e+15
75%,7.499988e+15
max,9.999858e+15


### Preprocess FrequentFlyerForum-Profiles.json

In [58]:
def prepare_passenger_data(passenger: dict, passenger_flight: dict):
    passenger_name = passenger['Real Name']
    passenger_flight_data = dict()
    passenger_flight_data.update(passenger_flight)
    passenger_flight_data.update(
        {
            "PassengerNickName": passenger['NickName'],
            "PassengerDocument": passenger['Travel Documents'][0]['Passports'],
            "PassengerSex": passenger['Sex'],
            "PassengerFirstName": passenger_name['First Name'],
            "PassengerLastName": passenger_name['Last Name'],
            "DepartureCountry": passenger_flight['Departure']['Country'],
            "DepartureCity": passenger_flight['Departure']['City'],
            "DepartureAirport": passenger_flight['Departure']['Airport'],
            "ArrivalCountry": passenger_flight['Arrival']['Country'],
            "ArrivalCity": passenger_flight['Arrival']['City'],
            "ArrivalAirport": passenger_flight['Arrival']['Airport'],
        })
    del passenger_flight_data['Departure']
    del passenger_flight_data['Arrival']
    return passenger_flight_data


flights = []

with open('data/FrequentFlyerForum-Profiles.json') as frequent_flyer_forum_file:
    passengers_data = json.load(frequent_flyer_forum_file)["Forum Profiles"]

    for passenger in passengers_data:
        passenger_flights = passenger['Registered Flights']
        passenger_loyalty_programs = passenger['Loyality Programm']
        for passenger_flight in passenger_flights:
            flights.append(prepare_passenger_data(passenger, passenger_flight))

flights_df = pd.DataFrame(flights)

In [61]:
flights_df.rename(columns={"Date": "DepartureDate", "Codeshare": "CodeShare", "Flight": "FlightNumber"}, inplace=True)
flights_df = flights_df.reindex(sorted(flights_df.columns), axis=1)
del flights_df["PassengerDocument"]

flights_df.head(5)
flights_df.info()
flights_df.describe()

,ArrivalAirport,ArrivalCity,ArrivalCountry,CodeShare,DepartureAirport,DepartureCity,DepartureCountry,DepartureDate,FlightNumber,PassengerFirstName,PassengerLastName,PassengerNickName,PassengerSex
0,PIT,Pittsburgh PA,United States of,False,BOS,Boston MA,United States of,2017-03-04,DL3377,None,None,FrequentFlyer0,Male
1,DTW,Detroit MI,United States of,False,GRB,Green Bay WI,United States of,2017-07-21,DL3630,None,None,FrequentFlyer0,Male
2,LGA,New York NY,United States of,False,PIT,Pittsburgh PA,United States of,2017-03-22,DL5361,None,None,FrequentFlyer0,Male
3,WAW,Warsaw,Poland,True,SVO,Moscow,Russian Federation,2017-01-04,SU3702,None,None,FrequentFlyer0,Male
4,ATL,Atlanta GA,United States of,False,XNA,Fayetteville AR,United States of,2017-08-08,DL3412,None,None,FrequentFlyer0,Male


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308987 entries, 0 to 1308986
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   ArrivalAirport      1308987 non-null  object
 1   ArrivalCity         1308987 non-null  object
 2   ArrivalCountry      1308987 non-null  object
 3   CodeShare           1308987 non-null  bool  
 4   DepartureAirport    1308987 non-null  object
 5   DepartureCity       1308987 non-null  object
 6   DepartureCountry    1308987 non-null  object
 7   DepartureDate       1308987 non-null  object
 8   FlightNumber        1308987 non-null  object
 9   PassengerFirstName  657755 non-null   object
 10  PassengerLastName   657755 non-null   object
 11  PassengerNickName   1308987 non-null  object
 12  PassengerSex        985901 non-null   object
dtypes: bool(1), object(12)
memory usage: 121.1+ MB


,ArrivalAirport,ArrivalCity,ArrivalCountry,CodeShare,DepartureAirport,DepartureCity,DepartureCountry,DepartureDate,FlightNumber,PassengerFirstName,PassengerLastName,PassengerNickName,PassengerSex
count,1308987,1308987,1308987,1308987,1308987,1308987,1308987,1308987,1308987,657755,657755,1308987,985901
unique,1129,1093,182,2,1129,1093,182,366,24005,200,1006,66592,2
top,SVO,Moscow,United States of,False,SVO,Moscow,Russian Federation,2017-05-10,SU4606,KRISTINA,CHERNYKH,FrequentFlyer60655,Male
freq,133633,137085,227462,1037438,133232,139183,279299,6629,2917,4023,1320,42,494134


### Write date to disk

In [62]:
boarding_data.to_csv("preprocessed/BoardingData.csv", index=False)
sirena_export.to_csv("preprocessed/SirenaExport.csv", index=False)
flights_df.to_csv("preprocessed/FrequentFlyerForumFlights.csv", index=False)